This notebook filters a model fine tuning dataset such that the the output text is a subset of the input data.  While we are using an LLM the output is essentially an extractive model.

The output was used to produce model 
https://wandb.ai/moso/tab_grouping/runs/ti3mvxgh?nw=nwuserrrando 
https://huggingface.co/Mozilla/smart-tab-topic/commit/4bd8ebe87ab4d719d2a58f263a3284eb2cbeeea9

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from bertopic import BERTopic

In [6]:
df = pd.read_csv("./test_data/topic_fine_tuning_data_guided__02_11_processed.csv")

In [7]:
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input_titles,input_keywords,output,assigned_topic,Hint,hint_guided_output,output_orig
0,0,0,8,8,Surprising Food Trends You Need to Know in 202...,"food,cooking,2023",Culinary Trends 202,0,Food,Culinary Trends 202,2023 Culinary Trends
1,1,1,34,34,Meal Prep for the Week: Save Time and Eat Heal...,"advanced,supplies,save",Home Essentials,0,Home,Home Essentials,Lifestyle Essentials
2,2,2,101,101,Easy One-Pot Pasta - Allrecipes\n50 Easy Dinne...,"easy,50,allrecipes",Easy Recipes,0,Recipes,Easy Recipes,Easy Recipes
3,3,3,163,163,Master of Malt: Online Shop for Whisky and Oth...,"wine,store,spirits",Liquor Stores,0,Spirits,Liquor Stores,Wine & Spirits Store
4,4,4,181,181,Bizarre Foods with Andrew Zimmern | Travel Cha...,"2023,travel,channel",Travel,0,NaN,Travel and Lifestyle Shows,Travel & Home Shows
...,...,...,...,...,...,...,...,...,...,...,...
2544,2544,2544,2179,2179,"How the Inkas Governed, Thrived and Fell Witho...",NaN,History,97,NaN,History,Inca Civilization
2545,2545,2545,1901,1901,How to get better at remembering | Psyche Guides,NaN,Self-Improvement,98,NaN,Self-Improvement,Memory Improvement
2546,2546,2546,2028,2028,GinkgoNotes: Never forget what you've learned,NaN,Note-taking Tools,98,NaN,Note-taking Tools,Learning Tools
2547,2547,2547,2355,2355,12 Things Gen X Was Taught In School That Woul...,NaN,Generational Differences,98,NaN,Generational Differences,Generational Perspectives


In [14]:
df = df.fillna("")

In [39]:
def is_output_subset(row):
    if row.input_titles.lower().find(row.hint_guided_output.lower()) >= 0:
        return row.hint_guided_output
    if row.input_keywords.lower().find(row.hint_guided_output.lower()) >= 0:
        return row.hint_guided_output
    split_string = row.hint_guided_output.split()
    if len(split_string) > 0:
        single_word_output = split_string[0]
        if row.input_titles.lower().find(single_word_output.lower()) >= 0:
            return single_word_output
        if row.input_keywords.lower().find(single_word_output.lower()) >= 0:
           return single_word_output
    return None

In [40]:
df["extractive_phrase"] = df.apply(is_output_subset, axis=1)

In [41]:
len(df[df["is_extractive"]==True])

1755

In [37]:
len(df)

2549

In [62]:
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input_titles,input_keywords,output,assigned_topic,Hint,hint_guided_output,output_orig,is_extractive,extractive_phrase
0,0,0,8,8,Surprising Food Trends You Need to Know in 202...,"food,cooking,2023",Culinary Trends 202,0,Food,Culinary Trends 202,2023 Culinary Trends,False,None
1,1,1,34,34,Meal Prep for the Week: Save Time and Eat Heal...,"advanced,supplies,save",Home Essentials,0,Home,Home Essentials,Lifestyle Essentials,True,Home
2,2,2,101,101,Easy One-Pot Pasta - Allrecipes\n50 Easy Dinne...,"easy,50,allrecipes",Easy Recipes,0,Recipes,Easy Recipes,Easy Recipes,True,Easy
3,3,3,163,163,Master of Malt: Online Shop for Whisky and Oth...,"wine,store,spirits",Liquor Stores,0,Spirits,Liquor Stores,Wine & Spirits Store,True,Liquor
4,4,4,181,181,Bizarre Foods with Andrew Zimmern | Travel Cha...,"2023,travel,channel",Travel,0,,Travel and Lifestyle Shows,Travel & Home Shows,True,Travel
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544,2544,2544,2179,2179,"How the Inkas Governed, Thrived and Fell Witho...",,History,97,,History,Inca Civilization,False,None
2545,2545,2545,1901,1901,How to get better at remembering | Psyche Guides,,Self-Improvement,98,,Self-Improvement,Memory Improvement,False,None
2546,2546,2546,2028,2028,GinkgoNotes: Never forget what you've learned,,Note-taking Tools,98,,Note-taking Tools,Learning Tools,False,None
2547,2547,2547,2355,2355,12 Things Gen X Was Taught In School That Woul...,,Generational Differences,98,,Generational Differences,Generational Perspectives,False,None


In [66]:
df = df[df.extractive_phrase.notna()]

In [67]:
df["output"] = df.extractive_phrase

In [68]:
df.to_csv("./test_data/topic_fine_tuning_data_extractive_2_15.csv")